In [1]:
import yaml
import numpy as np

from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import directed_hausdorff

from data.dataset import PolygonDataset
from train.trainer import Trainer
from model.nn import build_model
from torchinfo import summary

## Datasets

In [2]:
with open('cfg/gae.yaml', 'r') as f:
    cfg = yaml.safe_load(f)  
    
import pandas as pd
osm_df = pd.read_pickle(cfg['osm'])
melb_df = pd.read_pickle(cfg['melb'])
glyph_df = pd.read_pickle(cfg['test'])

osm_df_o = osm_df[osm_df.trans == 'o'].reset_index(drop=True)    
osm_df_r = osm_df[osm_df.trans == 'r'].reset_index(drop=True)  

glyph_df_o = glyph_df[glyph_df.trans == 'o'].reset_index(drop=True)
glyph_df_r = glyph_df[glyph_df.trans == 'r'].reset_index(drop=True)
glyph_df_sk = glyph_df[glyph_df.trans == 'sk'].reset_index(drop=True)
glyph_df_sc = glyph_df[glyph_df.trans == 'sc'].reset_index(drop=True)

glyph_set = PolygonDataset(glyph_df)
glyph_set_o = PolygonDataset(glyph_df_o)
glyph_set_r = PolygonDataset(glyph_df_r)
glyph_set_sk = PolygonDataset(glyph_df_sk)
glyph_set_sc = PolygonDataset(glyph_df_sc)

osm_set_o = PolygonDataset(osm_df_o)
osm_set_r = PolygonDataset(osm_df_r)
melb_set = PolygonDataset(melb_df)

datasets = {
            'glyph_set_o':glyph_set_o,
            'glyph_set_r':glyph_set_r,
            'glyph_set_sk': glyph_set_sk,
            'glyph_set_sc': glyph_set_sc,
            'osm_set_o': osm_set_o, 
            'osm_set_r': osm_set_r,
            'melb_set': melb_set
            }

backbone = [
            'gcn', 
            'gin', 
            'edgc', 
            ]

ckpt = 'epoch100'

## Baseline

In [3]:

cfg['nn'] = 'gcn'
cfg['path'] = f'save/baseline/{cfg["nn"]}'

model = build_model(cfg=cfg)
trainer = Trainer(cfg=cfg) 
model = trainer.load_ckpt(model, 'epoch100')
print(summary(model))

metric = {}

for key, dataset in datasets.items():
    zs = []
    for data in dataset:
        data.to('cuda')
        z = model.encoder(data.pos, data.edge_index)
        zs.append(z.mean(0).detach().cpu().numpy())

    zs = np.stack(zs, axis=0)
    zs = zs[~np.isnan(zs).any(axis=1)]
    
    # NN neighbors
    nbrs = NearestNeighbors(n_neighbors=7, algorithm='auto').fit(zs)
    dist, idx = nbrs.kneighbors(zs)
    # Hausdorff dist    
    query_id = idx[:, 0]
    target_id = idx[:, 1:]
    haus_dist = []
    for id, tids in enumerate(target_id):
        qid = query_id[id]
        tem_dist = []
        for tid in tids: 
            dist = directed_hausdorff(dataset[qid].pos.cpu().numpy(), 
                                      dataset[tid].pos.cpu().numpy())
            tem_dist.append(dist[0])
        haus_dist.append(tem_dist)
        
    metric[key] = {'avg' : np.stack(haus_dist, axis=0).mean(axis=0).tolist(), 
                   'std' : np.stack(haus_dist, axis=0).std(axis=0).tolist()}
    
baseline = {'gcn' : metric}
baseline

Layer (type:depth-idx)                   Param #
GAE                                      --
├─GCNBlock: 1-1                          --
│    └─GCNConv: 2-1                      64
│    │    └─SumAggregation: 3-1          --
│    │    └─Linear: 3-2                  128
│    └─GCNConv: 2-2                      64
│    │    └─SumAggregation: 3-3          --
│    │    └─Linear: 3-4                  4,096
│    └─LayerNorm: 2-3                    128
│    └─PReLU: 2-4                        1
├─GCNBlock: 1-2                          --
│    └─GCNConv: 2-5                      64
│    │    └─SumAggregation: 3-5          --
│    │    └─Linear: 3-6                  4,096
│    └─GCNConv: 2-6                      2
│    │    └─SumAggregation: 3-7          --
│    │    └─Linear: 3-8                  128
│    └─LayerNorm: 2-7                    128
│    └─PReLU: 2-8                        1
Total params: 8,900
Trainable params: 8,900
Non-trainable params: 0


{'gcn': {'glyph_set_o': {'avg': [0.19545930450332405,
    0.23092810798822067,
    0.24691008325912162,
    0.2586500592474599,
    0.26518790039494555,
    0.2710368128246281],
   'std': [0.1547576213324943,
    0.15703656670187927,
    0.15928536981796354,
    0.16183854521106764,
    0.16064547171593602,
    0.1612253619846148]},
  'glyph_set_r': {'avg': [0.27547866712753305,
    0.302778669021704,
    0.31876750135143966,
    0.326271743658721,
    0.3363783832612631,
    0.3434518097156476],
   'std': [0.15479974733839313,
    0.1545914271524566,
    0.15556858144614083,
    0.15481991054239455,
    0.1547318991112567,
    0.15393950035231166]},
  'glyph_set_sk': {'avg': [0.2883627432489087,
    0.31168441267815283,
    0.32706383409359185,
    0.3323268831223962,
    0.3393498619020947,
    0.3434746668750807],
   'std': [0.14494592305654885,
    0.1463101630664732,
    0.1492182582708502,
    0.14685418697704597,
    0.14933515687013663,
    0.15022424506982632]},
  'glyph_set_s

### No Aug

In [4]:
no_aug = {}
for nn in backbone:
    cfg['nn'] = nn
    cfg['path'] = f'save/no_aug/{cfg["nn"]}'

    model = build_model(cfg=cfg)
    trainer = Trainer(cfg=cfg) 
    model = trainer.load_ckpt(model, ckpt)
    print(summary(model))

    metric = {}
    
    for key, dataset in datasets.items():
        zs = []
        for data in dataset:
            data.to('cuda')
            z = model.encoder(data.pos, data.edge_index)
            zs.append(z.mean(0).detach().cpu().numpy())

        zs = np.stack(zs, axis=0)
        zs = zs[~np.isnan(zs).any(axis=1)]
        
        # NN neighbors
        nbrs = NearestNeighbors(n_neighbors=7, algorithm='auto').fit(zs)
        dist, idx = nbrs.kneighbors(zs)
        # Hausdorff dist    
        query_id = idx[:, 0]
        target_id = idx[:, 1:]
        haus_dist = []
        for id, tids in enumerate(target_id):
            qid = query_id[id]
            tem_dist = []
            for tid in tids: 
                dist = directed_hausdorff(dataset[qid].pos.cpu().numpy(), 
                                          dataset[tid].pos.cpu().numpy())
                tem_dist.append(dist[0])
            haus_dist.append(tem_dist)

        metric[key] = {'avg' : np.stack(haus_dist, axis=0).mean(axis=0).tolist(), 
                       'std' : np.stack(haus_dist, axis=0).std(axis=0).tolist()}
        
    no_aug[nn] = metric
no_aug

Layer (type:depth-idx)                   Param #
GAE                                      --
├─GCNBlock: 1-1                          --
│    └─GCNConv: 2-1                      64
│    │    └─SumAggregation: 3-1          --
│    │    └─Linear: 3-2                  128
│    └─GCNConv: 2-2                      64
│    │    └─SumAggregation: 3-3          --
│    │    └─Linear: 3-4                  4,096
│    └─LayerNorm: 2-3                    128
│    └─PReLU: 2-4                        1
├─GCNBlock: 1-2                          --
│    └─GCNConv: 2-5                      64
│    │    └─SumAggregation: 3-5          --
│    │    └─Linear: 3-6                  4,096
│    └─GCNConv: 2-6                      2
│    │    └─SumAggregation: 3-7          --
│    │    └─Linear: 3-8                  128
│    └─LayerNorm: 2-7                    128
│    └─PReLU: 2-8                        1
Total params: 8,900
Trainable params: 8,900
Non-trainable params: 0
Layer (type:depth-idx)                  

{'gcn': {'glyph_set_o': {'avg': [0.18398993276870482,
    0.21943762812249368,
    0.23633359273799265,
    0.24616076299655434,
    0.2564921240576082,
    0.2609511508892518],
   'std': [0.15001076743115174,
    0.15698272752993594,
    0.15641969114712254,
    0.15912251364487195,
    0.16155360082540823,
    0.16163709732472628]},
  'glyph_set_r': {'avg': [0.26570491880890007,
    0.29530679001857385,
    0.31258078443138815,
    0.3253364341197949,
    0.3329171082586474,
    0.3410522212769819],
   'std': [0.15663077659896948,
    0.15782914526106517,
    0.1615121669212982,
    0.16322875990288438,
    0.16363146728402156,
    0.1657417099764744]},
  'glyph_set_sk': {'avg': [0.2757448443106998,
    0.3009222936511208,
    0.3176113563842079,
    0.32502946967026125,
    0.3326765423358635,
    0.3379822494065331],
   'std': [0.14538544960053632,
    0.1481605418691219,
    0.15131732896045655,
    0.15178663568438105,
    0.15156954261863562,
    0.1542433962151958]},
  'glyph_s

### Aug

In [6]:
aug = {}
for nn in backbone:
    cfg['nn'] = nn
    cfg['path'] = f'save/aug/{cfg["nn"]}'

    model = build_model(cfg=cfg)
    trainer = Trainer(cfg=cfg) 
    model = trainer.load_ckpt(model, ckpt)
    print(summary(model))

    metric = {}
    
    for key, dataset in datasets.items():
        zs = []
        for data in dataset:
            data.to('cuda')
            z = model.encoder(data.pos, data.edge_index)
            zs.append(z.mean(0).detach().cpu().numpy())

        zs = np.stack(zs, axis=0)
        zs = zs[~np.isnan(zs).any(axis=1)]
        
        # NN neighbors
        nbrs = NearestNeighbors(n_neighbors=7, algorithm='auto').fit(zs)
        dist, idx = nbrs.kneighbors(zs)
        # Hausdorff dist    
        query_id = idx[:, 0]
        target_id = idx[:, 1:]
        haus_dist = []
        for id, tids in enumerate(target_id):
            qid = query_id[id]
            tem_dist = []
            for tid in tids: 
                dist = directed_hausdorff(dataset[qid].pos.cpu().numpy(), 
                                          dataset[tid].pos.cpu().numpy())
                tem_dist.append(dist[0])
            haus_dist.append(tem_dist)
        
        metric[key] = {'avg' : np.stack(haus_dist, axis=0).mean(axis=0).tolist(), 
                       'std' : np.stack(haus_dist, axis=0).std(axis=0).tolist()}
        
    aug[nn] = metric
aug

Layer (type:depth-idx)                   Param #
GAE                                      --
├─GCNBlock: 1-1                          --
│    └─GCNConv: 2-1                      64
│    │    └─SumAggregation: 3-1          --
│    │    └─Linear: 3-2                  128
│    └─GCNConv: 2-2                      64
│    │    └─SumAggregation: 3-3          --
│    │    └─Linear: 3-4                  4,096
│    └─LayerNorm: 2-3                    128
│    └─PReLU: 2-4                        1
├─GCNBlock: 1-2                          --
│    └─GCNConv: 2-5                      64
│    │    └─SumAggregation: 3-5          --
│    │    └─Linear: 3-6                  4,096
│    └─GCNConv: 2-6                      2
│    │    └─SumAggregation: 3-7          --
│    │    └─Linear: 3-8                  128
│    └─LayerNorm: 2-7                    128
│    └─PReLU: 2-8                        1
Total params: 8,900
Trainable params: 8,900
Non-trainable params: 0
Layer (type:depth-idx)                  

{'gcn': {'glyph_set_o': {'avg': [0.17688330002417837,
    0.20861677184281757,
    0.22573735171966625,
    0.23598882111740313,
    0.24176511333774858,
    0.24828384649444304],
   'std': [0.14084869775694484,
    0.14420852487067498,
    0.14713552640831806,
    0.14879340170119726,
    0.14759989317208524,
    0.14990409878111047]},
  'glyph_set_r': {'avg': [0.24615575163659434,
    0.27246552743835056,
    0.28685696841404096,
    0.29729434007091726,
    0.3040823954803873,
    0.3105559785183561],
   'std': [0.14006413693658815,
    0.1421075419083813,
    0.1419160230527725,
    0.14349156877127395,
    0.14384818302005836,
    0.14479183336917578]},
  'glyph_set_sk': {'avg': [0.2577860479692769,
    0.2811983245052385,
    0.2940226649243059,
    0.30257905128327106,
    0.3088771636738263,
    0.31428522531711767],
   'std': [0.130240465938511,
    0.13250703703275674,
    0.1347942703663165,
    0.13570795173274147,
    0.13795427652177927,
    0.1371840403996605]},
  'glyph